# EXPLORACIÓN Y LIMPIEZA DEL DATASET

- Cargamos **librerías, funciones y el data set** que vamos a usar para la limpieza 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
#ORDENAR - esta es la importacion del archivo de las funciones de limpieza que me he creado
import sys
sys.path.append("src/")
from src.cleaning_functions import *

In [3]:
ds = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

- Hacemos una copia del data set "just in case"...

In [4]:
df = ds.copy()

# EXPLORACIÓN DATA SET

![alt text](https://memegenerator.net/img/instances/40379228/let-me-take-a-look-at-this.jpg "Take a look")

In [5]:
df.shape #25.723 Lineas y 24 columnas

(25723, 24)

### Vamos a cambiar las opciones de pandas para poder ver todas las columnas del dataset mejor

In [6]:
pd.options.display.max_columns = None

In [7]:
df.sample(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
10473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5685,1892.00.00,1892,1892.0,Provoked,AUSTRALIA,Torres Strait,Badu Island,Dress diving,Mr. A. Rotaman,M,NaN,FATAL Bitten in two by shark that he m...,Y,NaN,NaN,"G.P. Whitley, p.259",1892.00.00-Rotaman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1892.00.00,1892.00.00,618.0,NaN,NaN
2479,1993.08.19,19-Aug-1993,1993.0,Unprovoked,USA,Hawaii,"Paukukalo, Maui","Surfing, paddling seawards",Reggie Williams,M,NaN,Abrasions & board bitten,N,07h05,NaN,"G. Balazs; T. Allen, p.117; Hawaii Department ...",1993.08.19.a-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1993.08.19,1993.08.19,3824.0,NaN,NaN
21292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,2015.10.17.b,17-Oct-2015,2015.0,Invalid,USA,Hawaii,"Waikiki,",Surfing,male,M,32,Left foot bitten by eel,NaN,19h20,No shark involvement,"KHON2, 10/17/2015",2015.10.17.b.-Hawaii. pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.10.17.b,2015.10.17.b,5959.0,NaN,NaN
10899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7768,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6326,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [9]:
df.describe().T #Solo hay dos columnas creadas como numéricas

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


In [10]:
Nan_percentage = round(df.isna().sum()*100/len(df),2) #Calculo el porcentaje de valores NaN
Nan_percentage.sort_values(ascending = False)

Unnamed: 22               100.00
Unnamed: 23                99.99
Time                       88.54
Species                    86.53
Age                        86.51
Sex                        77.70
Activity                   77.62
Fatal (Y/N)                77.60
Location                   77.60
Area                       77.27
Name                       76.32
Country                    75.69
Injury                     75.61
Investigator or Source     75.57
Type                       75.52
Year                       75.51
Date                       75.50
pdf                        75.50
href formula               75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Case Number                66.17
dtype: float64

![alt text](https://miro.medium.com/max/413/0*Cir0TzUEkHMbb8QB "Cleaning data")

- Muchas columnas tienen demasiado porcentaje de NaN's por lo que podremos prescindir de ellas ya que no aportan información.
- Vamos a eliminar todas las filas que tienen NaN en todos sus campos

In [11]:
df.dropna(axis=0, inplace= True, how='all') #Elimino las filas que tienen todos los valores NaN

In [12]:
df.shape #Ahora tenemos 8.703 filas

(8703, 24)

- Actualizamos el porcentaje de NaN's de cada columna:

In [13]:
df.isnull().sum().apply(lambda x: x*100/df.shape[0]).sort_values(ascending=False) #Otro método parecido al anterior

Unnamed: 22               99.988510
Unnamed: 23               99.977019
Time                      66.126623
Species                   60.197633
Age                       60.117201
Sex                       34.080202
Activity                  33.838906
Location                  33.792945
Fatal (Y/N)               33.781455
Area                      32.816270
Name                      30.001149
Country                   28.162703
Injury                    27.909916
Investigator or Source    27.783523
Type                      27.634149
Year                      27.611169
href formula              27.599678
Date                      27.588188
pdf                       27.588188
href                      27.588188
Case Number.1             27.588188
Case Number.2             27.588188
original order            27.507756
Case Number                0.011490
dtype: float64

- Vamos a ver si hay alguna fila duplicada, en tal caso las eliminaremos

In [14]:
df.duplicated().sum() #2392 duplicados

2392

In [15]:
df.iloc[8698:8702] #Son filas con todo NaN salvo el case number, las eliminaremos también

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.drop_duplicates(inplace=True) #Elimino las filas duplicadas
df.reset_index(drop=True, inplace=True) #Reiniciamos el índice

- Eliminamos las columnas 'Unnamed: 22' y 'Unnamed: 23', ya vimos antes que eran prácticamente 100% NaN

In [17]:
df.drop(['Unnamed: 22','Unnamed: 23'], axis=1, inplace=True)

- Veámos como están escritos los nombres de las columnas

In [18]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

- Quitemos los espacios en blanco en los nombres de las columnas, capitalicemos los nombres y sustituyamos espacios por guiones bajos

In [19]:
df.columns = df.columns.str.strip().str.capitalize().str.replace(' ', '_')

# LIMPIEZA

![alt text](https://miro.medium.com/max/568/1*S1HH5F8PqWWcId9sb0L8og.jpeg "Dropna")

- Quitamos todas las filas y columnas que tengan todo NaN

In [ ]:
df.dropna(axis = 0, how = 'all', inplace = True)
df.dropna(axis = 1, how = 'all', inplace = True)

df.drop_duplicates(inplace=True)


df.shape #hemos quitado bastantes filas, realmente columnas no hemos limpiado directamente ya que Unnamed 22 y 23 tenian 1 y 2 datos

In [ ]:
df["Case Number"].sample(40)

In [ ]:
df[["Case Number", "Date","Investigator or Source",'Case Number.1', 'Case Number.2', 'original order']].sample(30)

- Echando una ojeada a estas columnas podemosdescartarlas ya que o están duplicadas o son datos que no nos van a aportar para nuestras hipótesis

In [ ]:
df[["pdf", "href","href formula"]].sample(20)

- Miramos uno de los links del href para ver contenido:

In [ ]:
df["href"][3513] 

https://sharkattackfile.net/spreadsheets/pdf_directory/1967.08.25-Casucci.pdf

In [ ]:
df["href formula"][6091]

http://sharkattackfile.net/spreadsheets/pdf_directory/1830.04.30-Bromwick.pdf


- Algunos estaban vacíos (iamgino que habrá cambiado en algo la dirección), otros contienen pdf´s con datos del ataque reportado: imagenes, recortes de periódico, fotos de la ubicación, etc....
Pero en este caso no voy a utilizarlos.

- Ojeadas las columnas podemos hacer **drop** de las que **no considero interesantes** para el caso:

'Case Number','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'

In [ ]:
df.drop(['Case Number','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'], axis=1, inplace=True)

In [ ]:
df.columns

### YEAR y DATE COLUMN

In [ ]:
df.Year.unique() #Vamos a ver si podemos rescatar algun año a través del contenido de la columna date

- Vemos años un poco raros, así que paso a verlos en detalle

In [ ]:
df[(df.Year == 0)]

- Hay 125 lineas con el df.Year == CERO pero en df.Date si que figuran datos, a ver que podemos rescatar de ahí.

In [ ]:
list(df.Date.unique())

- Mucho reported y mucho espacio en blanco:

In [ ]:
df.Date = df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(df.Date.unique())

- Aunque no salen en los **uniques** hay muchos Dates que salen como rangos, o datos de antes de Cristo.
- Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:

In [ ]:
salvables = df.loc[(df["Year"] == 0) & (df["Date"] != np.nan)]
salvables.shape

In [ ]:
salvables.sample(30)

- Llamamos a las funciones definidas en el **cleanin_functions.py**, mas concretamente a rescatar fechas que aplica 3 funciones secuencialmente donde coge por orden los que contienen BC, los que son fechas sueltas (tipo Before YYYY) y luego los intervalos de los cuales saca la media, para rellenar los datos de df.Year para esos valores de df.Dates  #TODO, mira tema de late´s y demas para meterlos tb

In [ ]:
df.Year = df.Date.apply(rescatar_fechas)

- Si miramos el dato primero de nuestra lista anterior vemos que:

In [ ]:
df["Date"][6228]

In [ ]:
df["Year"][6228]

In [ ]:
df["Date"][6265]

In [ ]:
df["Year"][6265]

In [ ]:
#pd.set_option('max_rows', None)

In [ ]:
df[["Date","Year"]].sample(10)

In [ ]:
#pd.set_option('max_rows', 20)

## AREA / LOCATION COLUMNS

- Ambas columnas son bastante imprecisas en cuanto a ubicacion, por lo que mejor voy a trabajar sobre la columna **country.**

In [ ]:
df[["Country","Area","Location"]].sample(10)

- Country voy a limpiar con una funcion metidad en cleaning_functios.py llamada paises. He cogido una lista de paises de Github que esta
metida en un CSV, lo que hace la funcion es mirar si la cadena correspondiente está en el CSV y sino le asigna NaN adema de un par de incorrecciones de la escritura de nombres que tambien he metido

In [ ]:
df.Country = df.Country.apply(paises)

In [ ]:
df.Country.notna().sum()

In [ ]:
#Esto se puede mejorar, para resultados sea...

# ACTIVITY COLUMN

In [ ]:
df.Activity.sample(30)

- La lista de actividades es amplia, vamos a agruparlas y filtrarlas a través de una función llamada actividad tambien contenida en cleaning_functions.py

In [ ]:
df.Activity = df.Activity.apply(actividad)
df.Activity.sample(30)

# COLUMNA INJURY

In [ ]:
list(df["Fatal (Y/N)"].unique())

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(30)

- El tema de valores distintos de los previsibles "Y"/ "N" ademas de categorizar la columna injury lo vamos a hacer a través de dos funciones

In [ ]:
df["Injury"] = df["Injury"].apply(lesiones)

In [ ]:
df['Fatal (Y/N)'] = df["Fatal (Y/N)"].apply(fatal)

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(10)

In [ ]:
#TODO age

# SPECIES COLUMN

In [ ]:
df.Species.sample(30)

- Para limpiar esto vamos a usar algo parecido a lo que hemos hecho con activity para categorizar y leer las cadenas de dentro.Usaré la lista de nombres de https://sharkattackfile.net/species.htm
 que es de donde viene nuestro dataframe además

In [ ]:
df.Species = df.Species.apply(species)

In [ ]:
df.Species.sample(30)

# Exportamos el data frame limpio a un CSV nuevo

In [ ]:
df.to_csv("src/attack_limpio.csv",index=False)

- La visualizacion continua en `analysis.ipynb` [📑](analysis.ipynb) 